In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, MaxPooling1D
from keras import metrics

from sklearn.model_selection import train_test_split
from keras.utils import np_utils

import DataProcessCNN as DP

Using TensorFlow backend.


In [2]:
#for subj_n in range(1,10):
    #load the data
#    subj_filename='C:/Users/lt/Desktop/DSW2018/PAMAP2_Dataset/Protocol/subject10'+str(subj_n)+'.dat'
#    dp=DP.dataprocess(subj_filename)
#    np.save('data'+str(subj_n)+'.npy', (DP.col_sublabels,dp.data_segmented))

In [3]:
datalabels=[]
for i in range(1,10):
    feature_names,datalabelsi=np.load('data'+str(i)+'.npy')
    datalabels.append(datalabelsi)

datalabels=np.vstack(datalabels)

In [4]:
X_all=datalabels[:,:,2:]
y_all=datalabels[:,0,1]

#convert activity id to lie within range(12)
act_list=np.unique(datalabels[:,0,1])
y_all2=np.nonzero(y_all[:,np.newaxis]==act_list)[1]

# convert labels to class matrix, one-hot-encoding
Y_all = np_utils.to_categorical(y_all2,12)
# split in train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X_all, Y_all, test_size=0.2)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train.reshape(len(X_train)*512,31))

X_trainnorm=(X_train-scaler.mean_)/scaler.scale_
X_testnorm=(X_test-scaler.mean_)/scaler.scale_

In [6]:
model = Sequential()
model.add(Convolution1D(16, 3, activation='relu', input_shape=(512,31)))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(12, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
model.fit(X_trainnorm, Y_train, validation_data=(X_testnorm,Y_test), epochs=10, batch_size=16,shuffle=True)

Train on 2664 samples, validate on 666 samples
Epoch 1/10
2664/2664 [==============================] - 2s 715us/step - loss: 1.2801 - acc: 0.5882 - val_loss: 0.5953 - val_acc: 0.7958
Epoch 2/10
2664/2664 [==============================] - 1s 563us/step - loss: 0.3488 - acc: 0.8949 - val_loss: 0.2765 - val_acc: 0.9219
Epoch 3/10
2664/2664 [==============================] - 2s 567us/step - loss: 0.1595 - acc: 0.9572 - val_loss: 0.1874 - val_acc: 0.9520
Epoch 4/10
2664/2664 [==============================] - 2s 581us/step - loss: 0.0820 - acc: 0.9771 - val_loss: 0.1538 - val_acc: 0.9565
Epoch 5/10
2664/2664 [==============================] - 2s 570us/step - loss: 0.0408 - acc: 0.9910 - val_loss: 0.1850 - val_acc: 0.9535
Epoch 6/10
2664/2664 [==============================] - 1s 563us/step - loss: 0.0278 - acc: 0.9936 - val_loss: 0.1307 - val_acc: 0.9655
Epoch 7/10
2664/2664 [==============================] - 2s 569us/step - loss: 0.0144 - acc: 0.9974 - val_loss: 0.1290 - val_acc: 0.9655
E